## 求職網非同步爬蟲

In [1]:
import requests as rq
from pyquery import PyQuery as pq
import re
import time
from datetime import datetime
import pandas as pd
import sqlalchemy as sa
from sqlite3 import OperationalError

from aiohttp import ClientSession
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [105]:
async def main(web):
    global df, keyword, end_page
    df = pd.DataFrame()
    async with ClientSession() as session:
        if web=='104':
            urls = [f'https://www.104.com.tw/jobs/search/?keyword={keyword}&order=1&page={str(i)}&jobsource=2018indexpoc&ro=0' for i in range(1,end_page+1)]
            tasks = [asyncio.create_task(fetch_104(url, session)) for url in urls]  # 建立任務清單
        if web=='cakeresume':
            urls = [f'https://www.cakeresume.com/jobs?q={keyword}&ref=navbar_quick_link_jobs&page={str(i)}' for i in range(1,end_page+1)]
            tasks = [asyncio.create_task(fetch_cakeresume(url, session)) for url in urls]  # 建立任務清單
        if web=='1111':
            urls = [f'https://www.1111.com.tw/search/job?ks={keyword}&fs=1&page={str(i)}' for i in range(1,end_page+1)]
            tasks = [asyncio.create_task(fetch_1111(url, session)) for url in urls]  # 建立任務清單
        await asyncio.gather(*tasks)  # 打包任務清單及執行

async def fetch_104(url, session):
    global df
    date, title, company, salary, link = [], [], [], [], []
    async with session.get(url) as response:  #非同步發送請求
        text = await response.text()
        doc = pq(text)
        for each in doc('.b-block__left').items():
            if each('.b-tit a'):
                if each('.b-tit__date').text():
                    date.append(each('.b-tit__date').text())
                else:
                    date.append('-')
                title.append(each('.b-tit .js-job-link').text())
                company.append(each('.b-list-inline.b-clearfix a').text())
                salary.append(each('.b-tag--default:nth-child(1)').text())
                link.append('https:'+each('.b-tit .js-job-link').attr("href"))
        
        crawl_at = datetime.now().strftime('%Y/%m/%d %H:%M:%S')
        data_dict = dict(zip(['刊登日期','職稱','公司名稱','薪資','webURL','關鍵字','爬蟲時間'],
                             [date,title,company,salary,link,keyword,crawl_at]))
        df = df.append(pd.DataFrame(data_dict))
        return df
    
async def fetch_cakeresume(url, session):
    global df
    title, link, company, job_desc, date, level, location, salary, tags = [],[],[],[],[],[],[],[],[]
    async with session.get(url) as response:  #非同步發送請求
        text = await response.text()
        doc = pq(text)
        for each in doc('.job.well-list-item.well-list-item-link').items():
            title.append(each('.job-title').text())
            link.append(each('.job-link').attr("href"))
            company.append(each('.page-name a').text())
            job_desc.append(each('.job-desc').text().replace('\n',''))
            date.append(each('.update-section').text())
            for item in each('.job-list-item-tags').items():
                level.append(item('.refinement-tags a').text())
                location.append(item('.info-section .location-section').text())
                salary.append(item('.info-section .job-salary-section').text())
                tags.append(item('.label').text())
        
        crawl_at = datetime.now().strftime('%Y/%m/%d %H:%M:%S')
        data_dict = dict(zip(['刊登日期','職稱','階級','描述','公司名稱','地點','薪資','webURL','關鍵字','爬蟲時間'],
                             [date,title,level,job_desc,company,location,salary,link,keyword,crawl_at]))
        df = df.append(pd.DataFrame(data_dict))
        return df
    
async def fetch_1111(url, session):
    global df
    title,update_date,company,category,location,salary,experience,education,job_desc,link = [],[],[],[],[],[],[],[],[],[]
    async with session.get(url) as response:  #非同步發送請求
        text = await response.text()
        doc = pq(text)
        for each in doc('.item__job').items():
            title.append(each('.item__job-position0.item__m--link').text())
            link.append(each('.item__job-info--link.item__job-position0--link').attr('href'))
            
            string = each('.item__job-organ a').attr('title')
            company.append(re.findall(r'《公司名稱》\w*',string)[0].split('《公司名稱》')[-1])
            category.append(re.findall(r'《行業類別》\w*',string)[0].split('《行業類別》')[-1])
            location.append(re.findall(r'《公司住址》\w*',string)[0].split('《公司住址》')[-1])
            
            date = each('.item__job-control-item.item__job-control-datechange')
            update_date.append(f'{date.attr("data-yyyy")}-{date.attr("data-mmdd")}')
            job_desc.append(each('.item__job-desc.item__job-desc-un_extension').attr('title'))
            
            salary.append(each('.item__job-prop-item.item__job-prop-salary').attr('aria-label'))
            experience.append(each('.item__job-prop-item.item__job-prop-experience').attr('aria-label'))
            education.append(each('.item__job-prop-item.item__job-prop-grade').attr('aria-label'))

        crawl_at = datetime.now().strftime('%Y/%m/%d %H:%M:%S')
        data_dict = dict(zip(['刊登日期','職稱','描述','經驗','學歷','公司名稱','公司類型','地點','薪資','webURL','關鍵字','爬蟲時間'],
                             [update_date,title,job_desc,experience,education,company,category,location,salary,link,keyword,crawl_at]))
        df = df.append(pd.DataFrame(data_dict))
        df = df.sort_values('刊登日期', ascending=False)
        return df
    
# 存檔
def save_to_sqlite(web, engine):
    # 讀sqlite裏頭的資料表名稱
    tables = pd.read_sql(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{web}'", engine)
    if tables.empty:
        df_raw = pd.DataFrame()
    else:
        df_raw = pd.read_sql(web, engine)
    df_raw = df_raw.append(df)
    df_raw.drop_duplicates(subset=['職稱','公司名稱'],keep='last',inplace=True)
    df_raw.to_sql(web, engine, if_exists="replace", index=False)
    print('存檔成功')
    
# 執行爬蟲
def start_crawling():
    global sqlite_path, web
    engine = sa.create_engine(f"sqlite:///{sqlite_path}")
    loop = asyncio.get_event_loop()  #建立事件迴圈(Event Loop)
    start = time.time()
    loop.run_until_complete(main(web))  #執行協程(coroutine)
    stop = time.time()
    print(f"歷經{stop-start}秒")
    save_to_sqlite(web, engine)

In [106]:
web = '1111'
# '104'
# 'cakeresume'
# '1111'
keyword = 'python'
end_page = 3
sqlite_path = './job.db'

In [107]:
start_crawling()

歷經1.743464708328247秒
存檔成功


In [108]:
df

,刊登日期,職稱,描述,經驗,學歷,公司名稱,公司類型,地點,薪資,webURL,關鍵字,爬蟲時間
16,2021-03-11,智慧製造程式設計工程師,1. 負責製造相關系統-應用分析與程式開發。\n\n2. 提升關鍵製程/設備/智動化運用-系...,2年工作經驗以上,專科以上,國聯機械實業股份有限公司,專用生產機械製造修配,高雄市岡山區本工東二路,"月薪 35,000~60,000元",https://www.1111.com.tw/job/92226033/,python,2021/03/11 09:47:44
0,2021-03-11,(RR)Python自動化軟體開發工程師-南港,1. 產品自動化驗證相關技術研究，驗証工具開發\n2. 建置和維護自動化平台\n3. 依據規...,3年工作經驗以上,"大學,碩士",神準科技股份有限公司,通訊機械器材相關,暫不提供地址,面議（經常性薪資4萬/月含以上）,https://www.1111.com.tw/job/92258025/,python,2021/03/11 09:47:45
3,2021-03-11,Python資深/工程師,1. 獨立開發Python應用程式與測試除錯\n2. 使用Python Web framew...,2年工作經驗以上,"專科,大學,碩士",春水堂科技娛樂股份有限公司,電腦軟體相關,暫不提供地址,面議（經常性薪資4萬/月含以上）,https://www.1111.com.tw/job/92257524/,python,2021/03/11 09:47:45
4,2021-03-11,【工業4.0／智慧製造】大數據分析工程師（Python）.,1. 大數據資料分析及應用\n2. 運用統計分析方法及機器學習提出適合演算模型\n3. 自動...,1年工作經驗以上,"大學,碩士",欣興電子股份有限公司,印刷電路板製造業,桃園市龜山區山鶯路,"月薪 32,000~70,000元",https://www.1111.com.tw/job/92259754/,python,2021/03/11 09:47:45
9,2021-03-11,C0225 軟體測試工程師/主管(昆山),工程師工作職掌:\n1.開發/維護產線生產測試軟體\n2.產品試產及問題分析\n3.具Pyt...,經驗不拘,"大學,碩士",仁寶電腦工業股份有限公司,電腦,台北市內湖區瑞光路,面議（經常性薪資4萬/月含以上）,https://www.1111.com.tw/job/92259249/,python,2021/03/11 09:47:45
...,...,...,...,...,...,...,...,...,...,...,...,...
2,2021-03-05,(RR)Python自動化軟體開發工程師-林口,職責:\n1. 自動化工具開發/測試與平台開發/維護\n2. 自動化測試結果產出Test R...,經驗不拘,"大學,碩士",神準科技股份有限公司,通訊機械器材相關,暫不提供地址,面議（經常性薪資4萬/月含以上）,https://www.1111.com.tw/job/92244787/,python,2021/03/11 09:47:45
6,2021-03-05,【110年研發替代役/預聘】Python自動化軟體開發工程師-南港,1. 產品自動化驗證相關技術研究，驗証工具開發\n2. 建置和維護自動化平台\n3. 依據規...,經驗不拘,"大學,碩士",神準科技股份有限公司,通訊機械器材相關,暫不提供地址,面議（經常性薪資4萬/月含以上）,https://www.1111.com.tw/job/92244827/,python,2021/03/11 09:47:45
8,2021-03-04,【中租超利士】GA數據分析師,1. 使用Python或R取得GA及Firebase網站及APP相關使用者行為數據，進行資料...,2年工作經驗以上,大學以上,中租迪和股份有限公司,其他金融,台北市內湖區瑞光路,面議（經常性薪資4萬/月含以上）,https://www.1111.com.tw/job/92242515/,python,2021/03/11 09:47:45
5,2021-02-06,研發類-無人機AI演算法工程師(CIRC),無人機巡檢應用\nA.AI 模型程式撰寫。（需要使用 Python / TensorFlow...,3年工作經驗以上,大學以上,中強光電股份有限公司,消費性電子產品製造,新竹市東區新竹科學工業園區力行路,面議（經常性薪資4萬/月含以上）,https://www.1111.com.tw/job/92214796/,python,2021/03/11 09:47:44


In [109]:
df[df.duplicated()]

,刊登日期,職稱,描述,經驗,學歷,公司名稱,公司類型,地點,薪資,webURL,關鍵字,爬蟲時間
